In [2]:
import neatmartinet as nm
from duplicatesuricate import Launcher,RecordLinker
from duplicatesuricate.evaluation import FuncEvaluationModel
import duplicatesuricate.preprocessing.companycleaning as cc

# Load Data

In [3]:
filename_input='f40_md2.csv'
filename_target='p11_md2.csv'
filename_out='f40_to_p11_' #+timeextract

In [4]:
df_target=pd.read_csv(filename_target,index_col=0)
#df_input = pd.read_csv(filename_input,index_col=0)

In [5]:
print(df_target.columns)
df_target.head()

Index(['KRAUS', 'LAND1', 'LIFNR', 'NAME1', 'NAME2', 'ORT01', 'PSTLZ', 'STCD1',
       'STCD2', 'STCEG', 'STRAS', 'SYSID', 'VBUND'],
      dtype='object')


,KRAUS,LAND1,LIFNR,NAME1,NAME2,ORT01,PSTLZ,STCD1,STCD2,STCEG,STRAS,SYSID,VBUND
companysysid,,,,,,,,,,,,,
P11_2003441,374521706,BE,2003441,EMERSON & CUMING MICROWAVE PRODUCTS,NV,Westerlo Belgium,2260,NaN,457356285,BE0457356285,Njverheidststraat 7a,P11,NaN
P11_2003442,28382311,BE,2003442,AFCEA EUROPE,"ASSOC. COMMUNICATIONS, ELECTRONICS",BRUSSELS,1030,NaN,NaN,BE425390431,RUE COLONEL BOURG 105,P11,NaN
P11_2003443,483517637,CH,2003443,APCO TECHNOLOGIES SA,NaN,Aigle,1860,NaN,NaN,NaN,Chemin de Champex 10,P11,NaN
P11_2003444,480213412,CH,2003444,MOWAG GMBH,NaN,KREUZLINGEN,8280,NaN,NaN,CH4403004218-8,UNTERSEESTRASSE 65,P11,NaN
P11_2003514,537369803,DE,2003514,ComConsult Akademie,NaN,Aachen,52076,NaN,NaN,DE154818876,Pascalstrasse 25,P11,NaN


In [6]:
query={'KRAUS': '537369803',
 'LAND1': 'DE',
 'LIFNR': 'ab',
 'NAME1': 'ComConsult',
 'ORT01': 'Aachen',
 'PSTLZ': '52076',
 'STCEG': 'DE154818876',
 'STRAS': 'Pascalstr.',
 'SYSID': 'df_target',
    'STCD1':None,
       'STCD2':None,
 'VBUND': None}
query=pd.Series(query,name='A1')
df_input=pd.DataFrame(query).transpose()

# Clean Data

In [7]:
coldict={'duns':'KRAUS',
 'name':'NAME1',
 'street':'STRAS',
        'countrycode':'LAND1'}
namecol=coldict['name']
streetcol=coldict['street']

id_cols=['STCD1','STCD2','STCEG','KRAUS','VBUND']
id_scores=[c+'_exactscore' for c in id_cols]
formatasstr=id_cols+['PSTLZ','LIFNR']

In [8]:
def cleanfunc(df,coldict):
    for c in formatasstr:
        df[c]=df[c].apply(nm.format_int_to_str)
    c = coldict['duns']
    df[c]=df[c].apply(cc.cleanduns)
    c = namecol
    df[c]=df[c].apply(nm.format_ascii_lower)
    df[c+'_wostopwords']=df[c].apply(cc.rmv_companystopwords)
    df[c+'_len']=df[c].apply(cc.name_len)
    c = streetcol
    df[c]=df[c].apply(nm.format_ascii_lower)
    df[c+'_wostopwords']=df[c].apply(cc.rmv_streetstopwords)
    return df

In [9]:
target_records=cleanfunc(df_target,coldict)

In [10]:
input_records=cleanfunc(df_input,coldict)
query=input_records.iloc[0]

# Start Deduplication

In [11]:
from duplicatesuricate import Launcher,RecordLinker
from duplicatesuricate.evaluation import FuncEvaluationModel

## Define filtering and evaluation model

## Define  evaluation model

In [12]:
#define first filter
filter_all_any={'all':[coldict['countrycode']],'any':id_cols}
#Define intermediate threshold before further filtering
int_threshold={streetcol+'_fuzzyscore':0.5}

#Define evaluation model, do not forget the used scores
streetscores=[streetcol+'_wostopwords_fuzzyscore',streetcol+'_fuzzyscore']
namesscores=[namecol+'_wostopwords_fuzzyscore',namecol+'_fuzzyscore']
def evaluation_func(row):
    has_sameid=row[id_scores].max()
    has_samestreet=row[streetscores].max()
    has_samename=row[namesscores].max()
    if has_sameid==1:
        if has_samestreet>0.5:
            return max(has_samestreet,has_samestreet)
        else:
            return min(has_samestreet,has_samename)
    else:
        if min(has_samename,has_samename)>0.8:
            return min(has_samename,has_samestreet)
        else:
            return 0
used_cols=streetscores+id_scores+namesscores
mymodel = FuncEvaluationModel(used_cols=used_cols,eval_func=evaluation_func)

## Test the model

In [13]:
#Test model
rl = RecordLinker(df=df_target,filterdict=filter_all_any,
                  intermediate_thresholds=int_threshold,evaluator=mymodel,decision_threshold=0.5)
y=rl.predict_proba(query)
print(y.loc[y>rl.decision_threshold])

companysysid
P11_2003514    0.69
Name: 1, dtype: float64


## Build the launcher and Launch deduplication

In [14]:
finalmodel = Launcher(input_records=df_input,target_records=df_target,linker=rl,cleanfunc=None)

In [15]:
matches=finalmodel.start_linkage()

starting deduplication at 2017-11-29 14:32:37.979627
1 of 1 deduplicated | time elapsed 6.762768 s
finished work at 2017-11-29 14:32:44.742775


In [16]:
displaycols=[namecol]+[streetcol]+id_cols
displayfuzzy=[namecol]+[streetcol]

In [17]:
results=finalmodel.format_results(matches,display=displaycols,fuzzy=displayfuzzy)
results.head()

,ix_source,ix_target,STCD2_source,STCD2_target,STCD1_source,STCD1_target,STCEG_source,STCEG_target,NAME1_source,NAME1_target,STRAS_source,STRAS_target,KRAUS_source,KRAUS_target,VBUND_source,VBUND_target,NAME1_score,STRAS_score
0,A1,P11_2003514,None,None,None,None,DE154818876,DE154818876,comconsult,comconsult akademie,pascalstr.,pascalstrasse 25,537369803,537369803,None,None,0.69,0.69


In [18]:
timeextract=pd.datetime.now().strftime('%Y%M%d-%H{}%m'.format('h'))
filename_out=filename_out+timeextract+'.xlsx'

In [19]:
results.to_excel(filename_out,index=True)